# Code of the Model

 ## Importing the necessary libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm
import warnings

# Suppressing warnings
warnings.filterwarnings("ignore")

# Setting the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## Data Transformations

In [ ]:
# Defining data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Loading the dataset with train transforms
data_dir = '/content/drive/MyDrive/Plant_leave_diseases_dataset_with_augmentation'
dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])

# Calculating dataset sizes and spliting it into train set and validation set
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Split dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Apply validation transformation to the validation dataset
val_dataset.dataset.transform = data_transforms['val']


## Dataset Loaders

In [ ]:
# Creating the data loaders of batch size 64
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

## Loading and Fine-Tuning the Model

In [ ]:
# Loading pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freezing all layers,here we will only train the final layers
for param in model.parameters():
    param.requires_grad = False

# Finetuning the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 39)

# Moving model to the selected device
model = model.to(device)

# Defining the loss function and optimizer for training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.0001)


## Training Loop

In [ ]:
# Training the model
num_epochs = 10

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    # Setting model to training mode
    model.train()

    running_loss = 0.0
    running_corrects = 0

    # Iterating over training data
    for inputs, labels in tqdm(train_loader, desc="Train Phase"):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Setting the Gradients to zero
        optimizer.zero_grad()

        # Forward pass of the model
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # Backward pass of the model
        loss.backward()
        optimizer.step()

        # Calculating the data
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset)

    print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print()


    print()

# Saving the trained model
torch.save(model, '/content/drive/MyDrive/plant_disease_classification.pth')


Epoch 1/10
----------


Train Phase: 100%|██████████| 769/769 [02:41<00:00,  4.76it/s]


Train Loss: 1.6314 Acc: 0.6857


Epoch 2/10
----------


Train Phase: 100%|██████████| 769/769 [02:41<00:00,  4.76it/s]


Train Loss: 0.6038 Acc: 0.9052


Epoch 3/10
----------


...

Train Loss: 0.1875 Acc: 0.9555


Epoch 9/10
----------


Train Phase: 100%|██████████| 769/769 [02:41<00:00,  4.76it/s]


Train Loss: 0.1736 Acc: 0.9586


Epoch 10/10
----------


Train Phase: 100%|██████████| 769/769 [02:41<00:00,  4.77it/s]


Train Loss: 0.1613 Acc: 0.9595




## Testing Loop

In [ ]:
# Testing the model
model.eval()
val_loss = 0.0
val_corrects = 0

all_labels = []
all_preds = []

with torch.no_grad():
    # Iteration over the validation set
    for inputs, labels in tqdm(val_loader, desc="Validation Phase"):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Getting Predictions from the model
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        val_loss += loss.item() * inputs.size(0)
        val_corrects += torch.sum(preds == labels.data)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Calculation of performance metrics
val_loss = val_loss / len(val_loader.dataset)
val_acc = val_corrects.double() / len(val_loader.dataset)

precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Validation Loss: {val_loss:.4f} Acc: {val_acc:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


Validation Phase: 100%|██████████| 193/193 [00:38<00:00,  5.04it/s]

Validation Loss: 0.1518 Acc: 0.9583
Precision: 0.9585
Recall: 0.9583
F1 Score: 0.9582


# Appendix

- Link of the Dataset
- Link of the Google Drive containing demo:
-Notebook link
-Model link


https://drive.google.com/file/d/1vMYnrpcmIK2t3HcaLnooKdkKJ78ydyAs/view?usp=sharing

https://colab.research.google.com/drive/1M3lbu0tWgDyaK945pY9qYcvYp4_50Ro8?usp=sharing


https://drive.google.com/file/d/16XW9e9WZoPQBVB7p6V9ZxXPjGPcU1LNz/view?usp=sharing


https://data.mendeley.com/datasets/tywbtsjrjv/1